## Chapter 3. 딕셔너리와 집합
### 3.1 일반적인 매핑형
표준 라이브러리에서 제공하는 매핑형은 모두 dict를 이용해서 구현하므로 키가 해시 가능해야 한다는 제한을 갖고 있다. 

NOTE : 해시 가능하다는 말의 의미
<p>수명 주기 동안 절대 변하지 않는 해시값을 갖고 있고(__hash__() 메서드가 필요) 다른 객체와 비교할 수 있으면(__eq__() 메서드가 필요), 객체를 해시 가능하다고 한다. 동일하다고 판단되는 객체는 반드시 해시값이 동일해야 한다.</p>

In [1]:
my_dict = {}
import collections
print(isinstance(my_dict, collections.abc.Mapping))
print(type(my_dict))

True
<class 'dict'>


In [2]:
tt = (1, 2, (30, 40))
print(hash(tt)) # 튜플은 들어 있는 항목들이 모두 해시 가능해야 해시 가능함

tf = (1, 2, frozenset([30, 40])) # frozenset은 언제나 해시 가능함
print(hash(tf))

tl = (1, 2, [30, 40]) # 리스트는 해시 불가
print(hash(tl))

8027212646858338501
985328935373711578


TypeError: unhashable type: 'list'

In [3]:
## dict을 구현하는 다양한 방법
a = dict(one=1, two=2, three=3)
b = {'one':1, 'two':2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'one':1, 'two':2, 'three':3})
a == b == c == d == e

True

### 3.2 지능형 딕셔너리

In [4]:
# 예제 3.1 지능형 딕셔너리 예제
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan')
]
country_code = {country: code for code, country in DIAL_CODES} # 쌍을 바꾸어서 country는 키, code는 값이 된다.
print(country_code)
print({code: country.upper() for country, code in country_code.items() if code < 66}) 

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62, 'Brazil': 55, 'Pakistan': 92, 'Bangladesh': 880, 'Nigeria': 234, 'Russia': 7, 'Japan': 81}
{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}


### 3.3 공통적인 매핑 메서드
[표 3.1]은 dict와 dict의 변형 중 가장 널리 사용되는 collections.defaultdict와 collections.OrderedDict 클래스가 구현하는 메서드를 보여준다. (p.114 참조)
<p>이 중 update(m, [\**kargs]) 메서드가 첫 인수 m을 다루는 방식은 <a href="https://nesoy.github.io/articles/2018-02/Duck-Typing">duck typing</a>의 대표적인 사례이다. m이 keys()메서드를 갖고 있는지 확인한 후, 만약 메서드를 갖고 있으면 매핑이라고 간주한다. keys() 메서드가 없으면, update() 메서드는 m의 항목들이 (키, 값) 쌍으로 되어 있다고 간주하고 m을 반복한다. 따라서 매핑은 다른 매핑으로 초기화하거나 (키, 값) 쌍을 생성할 수 있는 반복형 객체로 초기화할 수 있다.</p>

### 3.3.1 존재하지 않는 키를 setdefault()로 처리하기
존재하지 않는 키를 처리할 때 dict.get()의 안좋은 사례 [예제 3-2]와 좋은 사례 [예제 3-4]를 비교해보자.
<p> ※ 정규식에 대한 설명 참조 : https://github.com/Gyubin/TIL/blob/master/Python/regular_expression.md</p>

In [5]:
# 참고. 정규식 이해하기
import re

text = '!@#abcde00 hello'
REGEX = re.compile('\w+')

result = REGEX.search(text) # search는 문자열 처음이 패턴과 맞지 않더라도 이후에도 맞는게 있는지 찾음.
if result:
    print('group: ', result.group())
    print('start: ', result.start())
    print('end: ', result.end())
    print('span: ', result.span())
else:
    print("No match") 
    
result = REGEX.match(text) # match는 전체 문자열에서 처음부터 바로 패턴이랑 일치하는지 찾음. 없는 경우 None을 리턴
if result:
    print('group: ', result.group())
    print('start: ', result.start())
    print('end: ', result.end())
    print('span: ', result.span())
else:
    print("No match")

find_result = REGEX.findall(text)
print(find_result)

iter_result = REGEX.finditer(text)
if iter_result:
    for mo in iter_result:
        print('group: ', mo.group())
        print('start: ', mo.start())
        print('end: ', mo.end())
        print('span: ', mo.span())    

group:  abcde00
start:  3
end:  10
span:  (3, 10)
No match
['abcde00', 'hello']
group:  abcde00
start:  3
end:  10
span:  (3, 10)
group:  hello
start:  11
end:  16
span:  (11, 16)


In [6]:
# 예제 3-2. dict.get()을 이용해서 인덱스에서 발생한 단어 목록을 가져와서 갱신
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다. """

import sys
import re

WORD_RE = re.compile(r'\w+') # 찾고자 하는 패턴을 등록. w는 [a-zA-Z]와 동일, +는 반복될 수 있음을 의미
index = {}

with open(r'/media/shyeon/PrivateData/_datasets/movielens-1m/README.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1): # iterator start index is 1
        for match in WORD_RE.finditer(line):
            # match object는 다음 코드에서와 같이 group(), start(), end(), span() 네 가지 메소드를 가지고 있다.
            # group은 매치된 첫 번째 문자열을 리턴하고, start는 전체 텍스트에서 그 매치된 문자열의 시작 인덱스,
            # end는 매치된 문자열의 마지막 인덱스+1을 리턴한다. span은 시작, 끝 인덱스를 튜플로 리턴한다.            
            word = match.group().upper()
            column_no = match.start() + 1
            location = (line_no, column_no)
            #Following is the syntax for dict.get() method
                #dict.get(key, default = None)
                    #Parameters
                    #key − This is the Key to be searched in the dictionary.
                    #default − This is the Value to be returned in case key does not exist.
            occurrences = index.get(word, []) # 없으면 빈 리스트를, 있으면 dict에 저장된 기존 리스트를 가져옴
            occurrences.append(location)
            index[word] = occurrences # 줄번호와 단어의 시점번호의 조합인 튜플을, 해당 단어를 키로 사용하는 딕셔너리 항목에 추가함
            #index.setdefault(word, []).append(location)

# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(115, 5)]
000 [(4, 23)]
040 [(5, 11)]
1 [(4, 21), (84, 25), (85, 26), (105, 5), (116, 5)]
10 [(42, 61), (125, 4)]
11 [(126, 4)]
1145 [(42, 64)]
12 [(125, 12), (127, 4)]
13 [(128, 4)]
14 [(129, 4)]
15 [(130, 4)]
16 [(131, 4)]
17 [(132, 4)]
18 [(105, 16), (106, 4), (106, 10), (133, 4)]
19 [(42, 9), (42, 29), (134, 4)]
1992 [(60, 1)]
1996 [(61, 51)]
2 [(87, 75), (117, 5)]
20 [(88, 26), (135, 4)]
2000 [(5, 55)]
2015 [(40, 42), (42, 22)]
209 [(4, 27)]
24 [(106, 13)]
25 [(107, 4), (107, 10)]
2827872 [(42, 69)]
3 [(4, 66), (118, 5)]
34 [(107, 13)]
35 [(108, 4), (108, 10)]
3952 [(85, 32)]
4 [(41, 76), (119, 5)]
44 [(108, 13)]
45 [(109, 4), (109, 10)]
49 [(109, 13)]
5 [(41, 73), (86, 25), (120, 5)]
50 [(110, 4), (110, 10)]
55 [(110, 13)]
56 [(111, 4), (111, 10)]
6 [(5, 9), (121, 5)]
6040 [(84, 31)]
7 [(122, 5)]
8 [(123, 5)]
9 [(124, 5)]
900 [(4, 68)]
A [(28, 13), (40, 30), (54, 35), (71, 39), (86, 23), (102, 24), (168, 38)]
ABOUT [(51, 21)]
ACADEMIC [(116, 10)]
ACCIDENTAL [(168, 53)]
ACCURAC

In [7]:
# 예제 3-4. dict.setdefault()를 사용해서 dict.get() 대신 단 한 줄로 구현
# 검색 횟수도 최대 3회에서 1회로 줄일 수 있음
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다. """

import sys
import re

WORD_RE = re.compile(r'\w+') # 찾고자 하는 패턴을 등록. w는 [a-zA-Z]와 동일, +는 반복될 수 있음을 의미
index = {}

with open(r'/media/shyeon/PrivateData/_datasets/movielens-1m/README.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1): # iterator start index is 1
        for match in WORD_RE.finditer(line):
            word = match.group().upper()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)

# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(115, 5)]
000 [(4, 23)]
040 [(5, 11)]
1 [(4, 21), (84, 25), (85, 26), (105, 5), (116, 5)]
10 [(42, 61), (125, 4)]
11 [(126, 4)]
1145 [(42, 64)]
12 [(125, 12), (127, 4)]
13 [(128, 4)]
14 [(129, 4)]
15 [(130, 4)]
16 [(131, 4)]
17 [(132, 4)]
18 [(105, 16), (106, 4), (106, 10), (133, 4)]
19 [(42, 9), (42, 29), (134, 4)]
1992 [(60, 1)]
1996 [(61, 51)]
2 [(87, 75), (117, 5)]
20 [(88, 26), (135, 4)]
2000 [(5, 55)]
2015 [(40, 42), (42, 22)]
209 [(4, 27)]
24 [(106, 13)]
25 [(107, 4), (107, 10)]
2827872 [(42, 69)]
3 [(4, 66), (118, 5)]
34 [(107, 13)]
35 [(108, 4), (108, 10)]
3952 [(85, 32)]
4 [(41, 76), (119, 5)]
44 [(108, 13)]
45 [(109, 4), (109, 10)]
49 [(109, 13)]
5 [(41, 73), (86, 25), (120, 5)]
50 [(110, 4), (110, 10)]
55 [(110, 13)]
56 [(111, 4), (111, 10)]
6 [(5, 9), (121, 5)]
6040 [(84, 31)]
7 [(122, 5)]
8 [(123, 5)]
9 [(124, 5)]
900 [(4, 68)]
A [(28, 13), (40, 30), (54, 35), (71, 39), (86, 23), (102, 24), (168, 38)]
ABOUT [(51, 21)]
ACADEMIC [(116, 10)]
ACCIDENTAL [(168, 53)]
ACCURAC

### 3.4 융통성 있게 키를 조회하는 매핑
검색할 때 키가 존재하는 않는 경우 이를 처리하기 위한 2가지 방법이 있다. 하나는 defaultdict를 사용하거나, 다른 하나는 dict 등의 매핑형을 상속해서 \_\_mission\_\_( ) 메서드를 추가하는 방법이다.

#### 3.4.1 defaultdict : 존재하지 않는 키의 또 다른 처리방법
[예제 3-5]는 [예제 3-4]의 setdefault를 사용하는 대신 defaultdict을 사용하여 문제를 해결한다. 

In [8]:
# [예제 3-5] defaultdict을 사용
# 예제 3-4. dict.setdefault()를 사용해서 dict.get() 대신 단 한 줄로 구현
# 검색 횟수도 최대 3회에서 1회로 줄일 수 있음
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다. """

import sys
import re
import collections

WORD_RE = re.compile(r'\w+') # 찾고자 하는 패턴을 등록. w는 [a-zA-Z]와 동일, +는 반복될 수 있음을 의미
index = collections.defaultdict(list) # 키가 없을 때 호출할 함수를 기본값으로 설정해 줌

with open(r'/media/shyeon/PrivateData/_datasets/movielens-1m/README.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1): # iterator start index is 1
        for match in WORD_RE.finditer(line):
            word = match.group().upper()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index[word].append(location) # 키 값이 없더라도 기본값인 list를 호출하여 빈 list를 대입한다.

# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(115, 5)]
000 [(4, 23)]
040 [(5, 11)]
1 [(4, 21), (84, 25), (85, 26), (105, 5), (116, 5)]
10 [(42, 61), (125, 4)]
11 [(126, 4)]
1145 [(42, 64)]
12 [(125, 12), (127, 4)]
13 [(128, 4)]
14 [(129, 4)]
15 [(130, 4)]
16 [(131, 4)]
17 [(132, 4)]
18 [(105, 16), (106, 4), (106, 10), (133, 4)]
19 [(42, 9), (42, 29), (134, 4)]
1992 [(60, 1)]
1996 [(61, 51)]
2 [(87, 75), (117, 5)]
20 [(88, 26), (135, 4)]
2000 [(5, 55)]
2015 [(40, 42), (42, 22)]
209 [(4, 27)]
24 [(106, 13)]
25 [(107, 4), (107, 10)]
2827872 [(42, 69)]
3 [(4, 66), (118, 5)]
34 [(107, 13)]
35 [(108, 4), (108, 10)]
3952 [(85, 32)]
4 [(41, 76), (119, 5)]
44 [(108, 13)]
45 [(109, 4), (109, 10)]
49 [(109, 13)]
5 [(41, 73), (86, 25), (120, 5)]
50 [(110, 4), (110, 10)]
55 [(110, 13)]
56 [(111, 4), (111, 10)]
6 [(5, 9), (121, 5)]
6040 [(84, 31)]
7 [(122, 5)]
8 [(123, 5)]
9 [(124, 5)]
900 [(4, 68)]
A [(28, 13), (40, 30), (54, 35), (71, 39), (86, 23), (102, 24), (168, 38)]
ABOUT [(51, 21)]
ACADEMIC [(116, 10)]
ACCIDENTAL [(168, 53)]
ACCURAC

#### 3.4.2 \_\_missing\_\_( ) 메서드
dict 클래스를 상속하고 \_\_missing\_\_( )를 정의하면 dict.\_\_getitem\_\_() 표준 메서드가 키를 발견할 수 없을 때 keyError를 발생시키지 않고 \_\_missing\_\_( ) 메서드를 호출한다. 
주의할 점은 \_\_missing\_\_( )은 d[k] 연산자와 같이 \_\_getitem\_\_( )을 사용할 때만 호출된다. 따라서 'in' 연산자를 구현하는 get( )이나 \_\_contain\_\_( )에는 영향을 주지 못한다. [예제 3-7]는 이점을 감안하여 get, \_\_contains\_\_( )에 d[k] 방식으로 조회하여 \_\_missing_\_( )을 호출하도록 구현하였다.

In [9]:
# [예제 3-7] 조회할 때 키를 문자열로 변환하는 StrKeyDict0
class StrKeyDict0(dict):
    
    def __missing__(self, key): # 일치하는 키가 없을 때 호출됨
        if isinstance(key, str): # 키가 문자형이면 정말 없은 것이므로 오류를 일으킴
            raise KeyError(key)
        return self[str(key)] # 키가 숫자형일 수 있으므로 문자형으로 변환하고 다시 get() 호출
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key): # in 연산자 (기존 Key 집합에 있는지 여부 확인)
        return key in self.keys() or str(key) in self.keys()

In [10]:
# [예제 3-6] 비문자열 키를 검색할 때 키를 발견하지 못하면 키를 문자열로 변환하여 재검색하는 StrKeyDict0
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

# d[key]
print(d[2])
print(d['2']) # 두 결과가 동일함
print(d[1]) # 키 값이 없는 경우 에러 출력

two
two


KeyError: '1'

In [12]:
# d.get(key)
print(d.get(2))
print(d.get(1)) # Key가 없을 때 default가 None이므로 None 출력

# 'in' 연산자
print(2 in d)
print(1 in d)

two
None
True
False


### 3.5 그 외의 매핑형

<p><li><b>collections.OrderedDict</b></li></p>
키를 삽입한 순서대로 유지한다. OrderedDict의 popitem()은 최근에 삽입한 항목을 커내지만 my_odict.popitem(last=True) 형태로 호출하면 처음 삽입한 항목을 꺼낸다.
<p><li><b>collections.ChainMap</b></li></p>
매핑형의 목록을 담고 있으며 한꺼번에 모두 검색할 수 있다. 각 매핑을 차례대로 검색하고, 그 중 하나에서라도 키가 검색되면 성공한다. 다음 예제는 변수를 조회하는 기본 규칙을 표현한 것이다. pylookup = ChainMap(locals(), globals(), vars(builtins))
<p><li><b>collections.Counter</b></li></p>
모든 키에 정수형 카운터를 갖고 있으며, 기존 키를 갱신하면 카운터가 늘어난다. 보통 해시 가능한 객체(키)나 항 항목이 여러 번 들어갈 수 있는 다중 집합(multiset)에서 객체의 수를 세기 위해 사용한다.

In [13]:
ct = collections.Counter("abracadabra")
print(ct)
ct.update("abracadabra")
print(ct)
ct.most_common(2)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 10, 'b': 4, 'r': 4, 'c': 2, 'd': 2})


[('a', 10), ('b', 4)]

<p><li><b>collections.UserDict</b></li></p>
표준 Dict처럼 작동하는 매핑을 순수 파이썬으로 구현한 클래스로 상속해서 사용하도록 설계되었다. 

### 3.6 UserDict 상속하기
UserDict은 dict을 상속하지 않고 내부에 실제 항목을 담고 있는 data라고 하는 dict 객체를 갖고 있다. 이렇게 구현함으로써 \_\_setitem\_\_( ) 등의 특수 메서드를 구현할 때 발생하는 원치 않는 재궈적 호출을 피할 수 있으며, \_\_contains\_\_() 메서드를 간단히 구현할 수 있다. [예제 3-7]과 비교해보자.

In [17]:
# [예제 3-8] 삽입, 갱신, 조회할 때 비문자열 키를 항상 문제열로 변환하는 StrKeyDict
import collections

class StrKeyDict(dict):
    
    def __missing__(self, key): # 일치하는 키가 없을 때
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key): # in 연산자
        return str(key) in self.data # 저장된 키가 모두 str 형이므로 self.data에서 바로 조회할 수 있다.
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item # 모든 키를 str형으로 변환

UserDict 클래스는 MutableMapping을 상속하므로 StrKeyDict는 결국 매핑의 모든 기능을 가지게 된다. 특히 다음 메서드는 상당히 유용하다.
<p><li><b>MutableMapping.update()</b></li></p>
직접 호출하거나 다른 매핑 혹은 (키,값) 쌍의 반복형 및 키워드 인수에서 객체를 로딩하기 위해 __init__()에 의해 사용될 수도 있다.이 메서드는 항목을 추가하기 위해 'self[key] = 값' 구문을 사용하므로 결국 서브클래스에서 구현한 __setitem__()을 호출하게 된다.
<p><li><b>Mapping.get()</b></li></p>
[예제 3-7]에서는 __getitem__()과 일치하는 결과를 가져오기 위해 get()을 직접 구현했지만, [예제 3-8]에서는 [예제 3-7]의 get()과 동일한 Mapping.get()을 상속받는다. 

### 3.7 불변 매핑
사용자가 실수로 매핑을 변경하지 못하도록 보장하고 싶은 경우를 위해 MappingProxyType이라는 래퍼 클래스를 제공하며 읽기 전용의 mappingproxy 객체를 반환한다.

In [18]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)
print(d_proxy[1]) # 읽기 접근은 가능하지만 
d_proxy[1] = 'X'  # 쓰기는 불가능 

{1: 'A'}
A


TypeError: 'mappingproxy' object does not support item assignment

In [19]:
d[2] = 'B'
print(d_proxy) # 동적인 d_proxy는 d의 변경사항을 바로 반영한다.

{1: 'A', 2: 'B'}


### 3.8 집합 이론
집합은 고유한 객체의 모음으로서 기본적으로 중복 항목을 제거한다. 집합 요소는 반드시 해시할 수 있어야 한다. set은 해시 가능하지 않지만 fronzenset은 해시 가능하므로 fronzenset이 set의 요소일 수 있다. 또한 집합형은 중위 연산자를 이용해서 합집합(a | b), 교집합(a & b), 차집합(a - b)의 집합 연산을 구현한다. 

#### 3.8.1 집합 리터럴
{1}, {1, 2} 등 구문은 수학적 표기법과 동일하지만, 공집합은 set()으로 표기한다. {}는 빈 딕셔너리이므로 주의한다. {1, 2, 3}과 같은 리터럴 집합 구문은 set([1, 2, 3])처럼 생성자를 호출하는 것 보다 더 빠르고 가독성도 좋다. 생성자를 명시적으로 호출하는 경우에는 파이썬이 생성자를 가져오기 위해 집합명을 검색하고, 리스트를 생성하고, 이 리스트를 생성자에 전달해야 하므로 더 느리다. 반면 리터럴 집합 구문은 SUILD_SET이라는 특수 바이트코드를 실행한다. 단, frozenset는 언제나 생성자를 호출해서 생성해야 한다.

In [20]:
from dis import dis
dis('{1}')
dis('set([1])') # 바이트코드 확인 시 더 길다는 것을 알 수 있음

dis('frozenset(range(10))')
print(frozenset(range(10)))

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE
  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE
  1           0 LOAD_NAME                0 (frozenset)
              2 LOAD_NAME                1 (range)
              4 LOAD_CONST               0 (10)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 RETURN_VALUE
frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})


#### 3.8.2 지능형 집합

In [15]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')} # 문자명 안에 'SIGN' 단어가 들어 있는 문자들의 집합을 생성한다.
                                                               # 문자명이 없는 값의 에러를 막기 위해 기본값 ''을 설정한다.

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

#### 3.8.3 집합 연산
[표 3-2] 수학 집합 연산 [표 3-3] 불리언형을 반환하는 비교 연산자와 메서드 참조 (p. 132~p.133)

### 3.9 dict과 set의 내부 구조
<li>파이썬 dict, set은 얼마나 효율적인가?</li>
<li>왜 순서가 없는가?</li>
<li>dict의 키와 set의 항목에 파이썬의 모든 객체를 사용할 수 없는 이유는 무엇인가?</li>
<li>dict의 키와 set의 항목의 순서가 왜 삽입 순서에 따라 달라지며, 객체 수명주기 동안 이 순서가 바뀔 수 있는 이유는 무엇인가?</li>
<li>딕셔너리와 집합을 반복하는 동안 항목을 추가하면 왜 안 될까?</li>

In [ ]:
#### 3.9.1 성능 실험
